In [1]:
spark

In [9]:
tweets_one = spark.read.option('header', True).csv('filtered-tweeter/')

In [10]:
tweets_one.show()

+--------------------+--------------------+----+-----+----+----+--------------------+-----------+----------------+-------------+--------------------+------------------+---------------------+-------------------+-----------+-----------+-------------+--------------+-------------------+-------------------+------+
|        company_name|              Sector|year|month| day|hour|                text|  user_name|user_screen_name|user_verified|user_followers_count|user_friends_count|user_favourites_count|user_statuses_count|quote_count|reply_count|retweet_count|favorite_count|           tweet_id|            user_id|Symbol|
+--------------------+--------------------+----+-----+----+----+--------------------+-----------+----------------+-------------+--------------------+------------------+---------------------+-------------------+-----------+-----------+-------------+--------------+-------------------+-------------------+------+
|           Visa Inc.|Information Techn...|2020|   11|   1|  15|RT 

In [28]:
companies = spark.read.option('header', True).csv('tweets/Company.csv')
company_tweets = spark.read.option('header', True).csv('tweets/Company_Tweet.csv')
tweets_two = spark.read.option('header', True).csv('tweets/Tweet.csv')

In [14]:
companies.show()

+-------------+------------+
|ticker_symbol|company_name|
+-------------+------------+
|         AAPL|       apple|
|         GOOG|  Google Inc|
|        GOOGL|  Google Inc|
|         AMZN|  Amazon.com|
|         TSLA|   Tesla Inc|
|         MSFT|   Microsoft|
+-------------+------------+



In [15]:
company_tweets.show()

+------------------+-------------+
|          tweet_id|ticker_symbol|
+------------------+-------------+
|550803612197457920|         AAPL|
|550803610825928706|         AAPL|
|550803225113157632|         AAPL|
|550802957370159104|         AAPL|
|550802855129382912|         AAPL|
|550802745737768960|         AAPL|
|550797494188142592|         AAPL|
|550797275786518528|         AAPL|
|550797272686923776|         AAPL|
|550796617444765696|         AAPL|
|550795512899960832|         AAPL|
|550795254102638593|         AAPL|
|550795167318700033|         AAPL|
|550795088821886976|         AAPL|
|550793298357391360|         AAPL|
|550793247669231616|         AAPL|
|550793108242198528|         AAPL|
|550791919815892992|         AAPL|
|550791232738590720|         AAPL|
|550790423888035840|         AAPL|
+------------------+-------------+
only showing top 20 rows



In [16]:
tweets_two.show()

+------------------+---------------+----------+--------------------+-----------+-----------+--------+
|          tweet_id|         writer| post_date|                body|comment_num|retweet_num|like_num|
+------------------+---------------+----------+--------------------+-----------+-----------+--------+
|550441509175443456|VisualStockRSRC|1420070457|lx21 made $10,008...|          0|          0|       1|
|550441672312512512|    KeralaGuy77|1420070496|Insanity of today...|          0|          0|       0|
|550441732014223360|    DozenStocks|1420070510|S&P100 #Stocks Pe...|          0|          0|       0|
|550442977802207232|   ShowDreamCar|1420070807|$GM $TSLA: Volksw...|          0|          0|       1|
|550443807834402816|   i_Know_First|1420071005|Swing Trading: Up...|          0|          0|       1|
|550443808606126081|  aaplstocknews|1420071005|Swing Trading: Up...|          0|          0|       1|
|550443809700851716|     iknowfirst|1420071005|Swing Trading: Up...|          0|  

In [29]:
tweets_two = tweets_two.join(company_tweets, ['tweet_id']).join(companies, ['ticker_symbol'])
tweets_two.show()

+-------------+-------------------+---------------+----------+--------------------+-----------+-----------+--------+------------+
|ticker_symbol|           tweet_id|         writer| post_date|                body|comment_num|retweet_num|like_num|company_name|
+-------------+-------------------+---------------+----------+--------------------+-----------+-----------+--------+------------+
|         TSLA|1000001732393553922|       talesftf|1527253966|Where did I write...|          0|          0|       0|   Tesla Inc|
|         TSLA|1000034965738983424|       burleyjc|1527261890|Question of the y...|          0|          0|       1|   Tesla Inc|
|        GOOGL|1000052963472265216|     exis10tial|1527266181|My advice: Don't ...|          0|          0|       1|  Google Inc|
|         AMZN|1000052963472265216|     exis10tial|1527266181|My advice: Don't ...|          0|          0|       1|  Amazon.com|
|         TSLA|1000068841559511041| NetflixAndLamp|1527269966|There are a lot o...|       

In [30]:
from pyspark.sql.functions import *

tweets_two = tweets_two.withColumn('date', from_unixtime('post_date')) \
                       .withColumn('year', year('date')) \
                       .withColumn('month', month('date')) \
                       .withColumn('day', dayofmonth('date')) \
                       .withColumn('hour', hour('date'))
tweets_two.show()

+-------------+-------------------+---------------+----------+--------------------+-----------+-----------+--------+------------+-------------------+----+-----+---+----+
|ticker_symbol|           tweet_id|         writer| post_date|                body|comment_num|retweet_num|like_num|company_name|               date|year|month|day|hour|
+-------------+-------------------+---------------+----------+--------------------+-----------+-----------+--------+------------+-------------------+----+-----+---+----+
|         TSLA|1000001732393553922|       talesftf|1527253966|Where did I write...|          0|          0|       0|   Tesla Inc|2018-05-25 10:12:46|2018|    5| 25|  10|
|         TSLA|1000034965738983424|       burleyjc|1527261890|Question of the y...|          0|          0|       1|   Tesla Inc|2018-05-25 12:24:50|2018|    5| 25|  12|
|        GOOGL|1000052963472265216|     exis10tial|1527266181|My advice: Don't ...|          0|          0|       1|  Google Inc|2018-05-25 13:36:21|2

In [33]:
company_metadata = spark.read.option('header', True).csv('s&p_500.csv')
company_metadata.show()

+------+--------------------+--------------------+
|Symbol|                Name|              Sector|
+------+--------------------+--------------------+
|   MMM|          3M Company|         Industrials|
|   AOS|     A.O. Smith Corp|         Industrials|
|   ABT| Abbott Laboratories|         Health Care|
|  ABBV|         AbbVie Inc.|         Health Care|
|  ABMD|             Abiomed|         Health Care|
|   ACN|           Accenture|Information Techn...|
|  ATVI| Activision Blizzard|Communication Ser...|
|  ADBE|          Adobe Inc.|Information Techn...|
|   AAP|  Advance Auto Parts|Consumer Discreti...|
|   AMD|Advanced Micro De...|Information Techn...|
|   AES|            AES Corp|           Utilities|
|   AFL|               Aflac|          Financials|
|     A|Agilent Technologies|         Health Care|
|   APD|Air Products & Ch...|           Materials|
|  AKAM| Akamai Technologies|Information Techn...|
|   ALK|    Alaska Air Group|         Industrials|
|   ALB|Albemarle Corpora...|  

In [35]:
tweets_two = tweets_two.join(company_metadata, tweets_two.ticker_symbol == company_metadata.Symbol)
tweets_two.show()

+-------------+-------------------+---------------+----------+--------------------+-----------+-----------+--------+------------+-------------------+----+-----+---+----+------+--------------------+--------------------+
|ticker_symbol|           tweet_id|         writer| post_date|                body|comment_num|retweet_num|like_num|company_name|               date|year|month|day|hour|Symbol|                Name|              Sector|
+-------------+-------------------+---------------+----------+--------------------+-----------+-----------+--------+------------+-------------------+----+-----+---+----+------+--------------------+--------------------+
|         TSLA|1000001732393553922|       talesftf|1527253966|Where did I write...|          0|          0|       0|   Tesla Inc|2018-05-25 10:12:46|2018|    5| 25|  10|  TSLA|          Tesla Inc.|Consumer Discreti...|
|         TSLA|1000034965738983424|       burleyjc|1527261890|Question of the y...|          0|          0|       1|   Tesla

In [36]:
print(tweets_one.columns)
print(tweets_two.columns)

['company_name', 'Sector', 'year', 'month', 'day', 'hour', 'text', 'user_name', 'user_screen_name', 'user_verified', 'user_followers_count', 'user_friends_count', 'user_favourites_count', 'user_statuses_count', 'quote_count', 'reply_count', 'retweet_count', 'favorite_count', 'tweet_id', 'user_id', 'Symbol']
['ticker_symbol', 'tweet_id', 'writer', 'post_date', 'body', 'comment_num', 'retweet_num', 'like_num', 'company_name', 'date', 'year', 'month', 'day', 'hour', 'Symbol', 'Name', 'Sector']


In [39]:
clean_tweets_one = tweets_one.withColumnRenamed('Symbol', 'ticker_symbol') \
                             .withColumnRenamed('Sector', 'sector') \
                             .withColumnRenamed('retweet_count', 'retweet_num') \
                             .withColumnRenamed('favorite_count', 'like_num') \
                             .withColumnRenamed('user_name', 'writer') \
                             .withColumnRenamed('text', 'body') \
                             .withColumnRenamed('reply_count', 'comment_num') \
                             .filter(col('writer') != '') \
                             .filter(col('ticker_symbol') != '') \
                             .filter(col('body') != '') \
                             .select('ticker_symbol', 'tweet_id', 'writer', 'body', 'comment_num', 'retweet_num', 'like_num', 'company_name', 'sector', 'year', 'month', 'day', 'hour', )


clean_tweets_one.show()


+-------------+-------------------+--------------------+--------------------+-----------+-----------+--------+------------+--------------------+----+-----+---+----+
|ticker_symbol|           tweet_id|              writer|                body|comment_num|retweet_num|like_num|company_name|              sector|year|month|day|hour|
+-------------+-------------------+--------------------+--------------------+-----------+-----------+--------+------------+--------------------+----+-----+---+----+
|            V|1322919099660337152|         ᴮᴱPushpita⁷|RT @choi_bts2: Th...|          0|          0|       0|   Visa Inc.|Information Techn...|2020|   11|  1|  15|
|            V|1322919137404874754|   JAE⁷ lost a tooth|RT @choi_bts2: Th...|          0|          0|       0|   Visa Inc.|Information Techn...|2020|   11|  1|  15|
|            V|1322919250647023618|ᴮᴱ vanya⁷ ◡̈ ✜ ᴇɴ...|RT @choi_bts2: Th...|          0|          0|       0|   Visa Inc.|Information Techn...|2020|   11|  1|  15|
|         

In [43]:
clean_tweets_two = tweets_two.select('ticker_symbol', 'tweet_id', 'writer', 'body', 'comment_num', 'retweet_num', 'like_num', 'company_name', 'sector', 'year', 'month', 'day', 'hour', )
clean_tweets_two.show()

+-------------+-------------------+---------------+--------------------+-----------+-----------+--------+------------+--------------------+----+-----+---+----+
|ticker_symbol|           tweet_id|         writer|                body|comment_num|retweet_num|like_num|company_name|              sector|year|month|day|hour|
+-------------+-------------------+---------------+--------------------+-----------+-----------+--------+------------+--------------------+----+-----+---+----+
|         TSLA|1000001732393553922|       talesftf|Where did I write...|          0|          0|       0|   Tesla Inc|Consumer Discreti...|2018|    5| 25|  10|
|         TSLA|1000034965738983424|       burleyjc|Question of the y...|          0|          0|       1|   Tesla Inc|Consumer Discreti...|2018|    5| 25|  12|
|        GOOGL|1000052963472265216|     exis10tial|My advice: Don't ...|          0|          0|       1|  Google Inc|Communication Ser...|2018|    5| 25|  13|
|         AMZN|1000052963472265216|     

In [45]:
tweets = clean_tweets_one.union(clean_tweets_two).distinct()
tweets.show()

+-------------+-------------------+--------------------+--------------------+-----------+-----------+--------+------------+--------------------+----+-----+---+----+
|ticker_symbol|           tweet_id|              writer|                body|comment_num|retweet_num|like_num|company_name|              sector|year|month|day|hour|
+-------------+-------------------+--------------------+--------------------+-----------+-----------+--------+------------+--------------------+----+-----+---+----+
|            V|1300774726432731137|              itz¹¹⁷|RT @KTH_Facts: Ta...|          0|          0|       0|   Visa Inc.|Information Techn...|2020|    9|  1|  12|
|            V|1306878046927028224|  📖TheLindoPress¹¹⁷|RT @pinkvilla: #B...|          0|          0|       0|   Visa Inc.|Information Techn...|2020|    9| 18|   8|
|            V|1321141264201453571|          ᴮᴱ TAM⁷ ⟭⟬|RT @hamacream: Re...|          0|          0|       0|   Visa Inc.|Information Techn...|2020|   10| 27|  17|
|          

In [46]:
print(clean_tweets_one.count())
print(clean_tweets_two.count())
print(tweets.count())

20948
4336445
4357380


In [47]:
tweets.coalesce(1).write.option('header', True).csv('tweets.csv')

In [49]:
stats = tweets.groupBy('ticker_symbol').count().orderBy(col('count').desc())
stats.show(1000)

+-------------+-------+
|ticker_symbol|  count|
+-------------+-------+
|         AAPL|1425080|
|         TSLA|1096977|
|         AMZN| 718734|
|         GOOG| 392572|
|         MSFT| 375735|
|        GOOGL| 327573|
|            V|   5447|
|          TXT|   2201|
|          ABT|   1693|
|           IT|   1373|
|          DFS|    408|
|          NOW|    364|
|           PM|    343|
|          ABC|    337|
|           HD|    335|
|          AFL|    332|
|          ICE|    302|
|          WBA|    273|
|          VAR|    260|
|           CL|    226|
|          IBM|    199|
|          PNW|    173|
|          CRM|    167|
|            A|    156|
|          PSA|    155|
|           EA|    152|
|            L|    151|
|            C|    149|
|           MA|    148|
|         ETSY|    137|
|          GIS|    137|
|           ED|    136|
|          GPS|    106|
|          AMD|     99|
|           GM|     99|
|           MO|     96|
|           IP|     93|
|          NEM|     92|
|           TT| 

In [51]:
stats.coalesce(1).write.option('header', True).csv('stats.csv')